In [102]:
# Load in relevant libraries, and alias where appropriate
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F

# Define relevant variables for the ML task
batch_size = 64
num_classes = 10
learning_rate = 0.001
num_epochs = 10

# Device will determine whether to run the training on GPU or CPU.
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


## Loading the dataset

In [103]:
#Loading the dataset and preprocessing
train_dataset = torchvision.datasets.MNIST(root = './data',
                                           train = True,
                                           transform = transforms.Compose([
                                                  transforms.Resize((32,32)),
                                                  transforms.ToTensor(),
                                                  transforms.Normalize(mean = (0.1307,), std = (0.3081,))]),
                                           download = True)


test_dataset = torchvision.datasets.MNIST(root = './data',
                                          train = False,
                                          transform = transforms.Compose([
                                                  transforms.Resize((32,32)),
                                                  transforms.ToTensor(),
                                                  transforms.Normalize(mean = (0.1325,), std = (0.3105,))]),
                                          download=True)


train_loader = torch.utils.data.DataLoader(dataset = train_dataset,
                                           batch_size = batch_size,
                                           shuffle = True)


test_loader = torch.utils.data.DataLoader(dataset = test_dataset,
                                           batch_size = batch_size,
                                           shuffle = True)

+ Đầu tiên, dữ liệu MNIST không thể được sử dụng như dữ liệu dành cho kiến ​​trúc LeNet5.Kiến trúc LeNet5 chấp nhận đầu vào là 32x32 và hình ảnh MNIST là 28x28.
+ Chúng ta có thể khắc phục điều này bằng cách thay đổi kích thước hình ảnh, chuẩn hóa chúng bằng cách sử dụng giá trị trung bình và độ lệch chuẩn được tính toán trước (có sẵn trực tuyến) và cuối cùng lưu trữ chúng dưới dạng tenxơ

## Lenet5 from scratch

In [104]:
class ConvNeuralNet(nn.Module):
    def __init__(self, num_classes):
        super(ConvNeuralNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 6, kernel_size=5, stride=1, padding=0),
            nn.BatchNorm2d(6),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(6, 16, kernel_size=5, stride=1, padding=0),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2))
        self.fc = nn.Linear(400, 120)
        self.relu = nn.ReLU()
        self.fc1 = nn.Linear(120, 84)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(84, num_classes)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        out = self.relu(out)
        out = self.fc1(out)
        out = self.relu1(out)
        out = self.fc2(out)
        return out

In [105]:
model = ConvNeuralNet( num_classes).to(device)

#Defining cost and optimizer
cost = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)  

In [106]:
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):  
        images = images.to(device)
        labels = labels.to(device)
            #Forward pass
        outputs = model(images)
        loss = cost(outputs, labels)
        	
        	# Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        		
        if (i+1) % 400 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
        		           .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

Epoch [1/10], Step [400/938], Loss: 0.1406
Epoch [1/10], Step [800/938], Loss: 0.0999
Epoch [2/10], Step [400/938], Loss: 0.0344
Epoch [2/10], Step [800/938], Loss: 0.0026
Epoch [3/10], Step [400/938], Loss: 0.0321
Epoch [3/10], Step [800/938], Loss: 0.0234
Epoch [4/10], Step [400/938], Loss: 0.0409
Epoch [4/10], Step [800/938], Loss: 0.0182
Epoch [5/10], Step [400/938], Loss: 0.0104
Epoch [5/10], Step [800/938], Loss: 0.0017
Epoch [6/10], Step [400/938], Loss: 0.0076
Epoch [6/10], Step [800/938], Loss: 0.1692
Epoch [7/10], Step [400/938], Loss: 0.0014
Epoch [7/10], Step [800/938], Loss: 0.0025
Epoch [8/10], Step [400/938], Loss: 0.0012
Epoch [8/10], Step [800/938], Loss: 0.0016
Epoch [9/10], Step [400/938], Loss: 0.0055
Epoch [9/10], Step [800/938], Loss: 0.1826
Epoch [10/10], Step [400/938], Loss: 0.0049
Epoch [10/10], Step [800/938], Loss: 0.0024


## Testing

In [107]:
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy of the network on the 10000 test images: {} %'.format(100 * correct / total))
	 

Accuracy of the network on the 10000 test images: 99.19 %
